In [1]:
import pandas as pd
import collections
import os
import zlib
import pickle

def deserialize(pz):
    if os.path.isfile(pz):
        compressed = open(pz, 'rb').read()
        pkl = zlib.decompress(compressed)
        tpl = pickle.loads(pkl)
        return tpl
    else:
        return []

lubm_classes_file = "../data/lubm1_intact/encoding/lubm_classes.pz"
lubm_properties_file = "../data/lubm1_intact/encoding/lubm_properties.pz"
lubm_properties_groups_file = "../data/lubm1_intact/encoding/lubm_properties_groups.pz"
tbox_file = "../data/lubm1_intact/encoding/tbox.nt"

lubm_classes = deserialize(lubm_classes_file)
lubm_properties = deserialize(lubm_properties_file)
lubm_properties_groups = deserialize(lubm_properties_groups_file)

data = []

for key, value in lubm_classes.items():
    data.append({"type": "class", "uri": key, "index": value})

for key, value in lubm_properties.items():
    data.append({"type": "property", "uri": key, "index": value})

for key, value in lubm_properties_groups.items():
    data.append({"type": "property_group", "uri": key, "index": value})


df = pd.DataFrame(data)
print(df[df.type=="class"].size,
      df[df.type=="property"].size,
      df[df.type=="property_group"].size)

#lubm_classes
df.shape

174 162 162


(166, 3)

# Creating a DF with clases, properties, property_groups
Even though relative - there is no relation between index

In [2]:
result_df = df.pivot_table(index='index', columns='type', values='uri', aggfunc='first')

In [3]:
result_df

type,class,property,property_group
index,,,
0,NaN,NaN,http://www.w3.org/2000/01/rdf-schema#seeAlso
1,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/1999/02/22-rdf-syntax-ns#_1
2,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...
3,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...
4,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
5,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...
6,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...
7,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...
8,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...,http://swat.cse.lehigh.edu/onto/univ-bench.owl...


In [4]:
df_class = df[df.type=="class"]
df_property = df[df.type=="property"]
df_property_group = df[df.type=="property_group"]

In [5]:
col_names=["subject", "predicate", "object"]
#df2 = pd.read_csv(tbox_file, delimiter=' ', names=col_names)
df2 = pd.read_csv(tbox_file, delimiter = ' ')
#df2['object'] = df2['object'].apply(lambda x: x.rstrip('.'))
df2 = df2.drop(df2.columns[3], axis = 1)
df2.columns = col_names
df2.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/lubm1_intact/encoding/tbox.nt'

# I want to see what "property types" are in the #type in df2 for "df1" values marked as "property"

In [ ]:
df_class

In [ ]:
df_property_group

# I will seperate only the subject #end part and compare

In [ ]:
df_property_copy = df_property.copy()

In [ ]:
df_property_copy.uri = df_property_copy.uri.str.extract(r'#(.+)')

In [ ]:
df_property_copy

In [ ]:
df1_property_list = df_property_copy.uri.to_list()

# DF 2 CLASSES

In [ ]:
df2

In [ ]:
df2_types = df2[df2.predicate.str.contains('#type')]

In [ ]:
df2_classes = df2[df2.object.str.contains('#Class')]

In [ ]:
df2_classes

# DF 2 PROPERTIES

In [ ]:
df2_types

# Compare property list from df1 to df2 subject, and get object "type"

In [ ]:
df1_property_list

In [ ]:
types = []
for x in range(len(df1_property_list)):
    types.append(df2_types[df2_types.subject.str.contains(df1_property_list[x])].object.str.extract(r'#(.+)'))

In [ ]:
types_2 = []
for type in types:
    types_2.append(type.to_string().strip())

In [ ]:
types_2

In [ ]:
df2_properties = df2_types.copy()

In [ ]:
df2_properties = df2_properties[df2_properties.object.str.contains("Property")].subject.unique()

In [ ]:
df2_properties

# DF2 SUB PROPERTIES

In [ ]:
df2_sub_properties = df2.copy()

In [ ]:
df2_sub_properties = df2_sub_properties[df2_sub_properties.predicate.str.contains("subPropertyOf")]

In [ ]:
df2_sub_properties

In [ ]:
import re

def get_lubm_graph_type(s):
    s = s.split('/')[-1]
    print(s)
    s = s.split('_')[-1]
    print(s)
    m = re.search("\d", s)
    print(m)
    return s[:m.start()]

In [ ]:
get_lubm_graph_type("HTTP_www.Department0.University0.edu_AssistantProfessor8_Publication1.nt")


In [ ]:
get_lubm_graph_type("C:/Users/martt/PycharmProjects/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")

In [8]:
from rdflib import Graph
import codecs
import os

In [34]:
input_graph = Graph()
input_file = os.path.abspath("/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu.nt")
input_file2 = os.path.abspath("/home/mart/git/NMT4RDFS/data/lubm1_intact/graphs_with_descriptions/HTTP_www.Department0.University0.edu_AssistantProfessor0.nt")
input_graph.parse(data=codecs.open(input_file, encoding="UTF-8").read(), format="nt")
graph = input_graph

In [35]:
triples_list = sorted(list(input_graph))[0]

In [36]:
from graph_words_encoder import GraphWordsEncoder
from prepare_data_utils import get_properties_dict, get_classes_dict, get_properties_groups

In [37]:
import configparser
import json
import logging

In [38]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

config = configparser.ConfigParser()
with open('../config.json', 'r') as f:
    config = json.load(f)

In [39]:
LUBM_ONTOLOGY_ENDPOINT = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT']
LUBM_ONTOLOGY_REPOSITORY = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_REPOSITORY']
LUBM_ONTOLOGY_ENDPOINT_TYPE = config['LUBM']['SPARQL_PARAMS']['ONTOLOGY_ENDPOINT_TYPE']
PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['PROPERTIES_QUERY']
QUERY_LIMIT = config['LUBM']['SPARQL_PARAMS']['QUERY_LIMIT']
PROPERTIES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_DICT_FILE']
CLASSES_QUERY = config['LUBM']['SPARQL_PARAMS']['CLASSES_QUERY']
LUBM_ENCODING_DIRECTORY = config['LUBM']['ENCODING_PARAMS']['ENCODING_DIRECTORY']
CLASSES_DICT_FILE = config['LUBM']['ENCODING_PARAMS']['CLASSES_DICT_FILE']
PROPERTIES_GROUPS_FILE = config['LUBM']['ENCODING_PARAMS']['PROPERTIES_GROUPS_FILE']
GRAPH_WORDS_ENCODER = config['LUBM']['ENCODING_PARAMS']['GRAPH_WORDS_ENCODER']
SUB_PROPERTIES_QUERY = config['LUBM']['SPARQL_PARAMS']['SUB_PROPERTIES_QUERY']
INPUT_GRAPHS_FOLDER = config['LUBM']['DATA']['INPUT_GRAPHS_FOLDER']
INFERENCE_GRAPHS_FOLDER = config['LUBM']['DATA']['INFERENCE_GRAPHS_FOLDER']
DATASET_ENCODING_FILE = config['LUBM']['ENCODING_PARAMS']['DATASET_ENCODING_FILE']
INPUT_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INPUT_GRAPH_WORDS_CATALOGUE_FILE']
INFERENCE_GRAPH_WORDS_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['INFERENCE_GRAPH_WORDS_CATALOGUE_FILE']
EMBEDDING_MATRIX_CATALOGUE_FILE = config['LUBM']['ENCODING_PARAMS']['EMBEDDING_MATRIX_CATALOGUE_FILE']
MAXIMUM_MATRIX_SIZE = config['LUBM']['ENCODING_PARAMS']['MAXIMUM_MATRIX_SIZE']
HOPE_EMBEDDING_SIZE = config['LUBM']['ENCODING_PARAMS']['HOPE_EMBEDDING_SIZE']

In [40]:
properties_dict = get_properties_dict(LUBM_ENCODING_DIRECTORY + PROPERTIES_DICT_FILE, PROPERTIES_QUERY,
                               LUBM_ONTOLOGY_REPOSITORY, LUBM_ONTOLOGY_ENDPOINT,
                               LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [41]:
classes_dict = get_classes_dict(LUBM_ENCODING_DIRECTORY + CLASSES_DICT_FILE, CLASSES_QUERY, LUBM_ONTOLOGY_REPOSITORY,
                            LUBM_ONTOLOGY_ENDPOINT,
                            LUBM_ONTOLOGY_ENDPOINT_TYPE, QUERY_LIMIT)

In [42]:
properties_group_dict = get_properties_groups(LUBM_ENCODING_DIRECTORY + PROPERTIES_GROUPS_FILE, SUB_PROPERTIES_QUERY,
                                 LUBM_ONTOLOGY_REPOSITORY,
                                 LUBM_ONTOLOGY_ENDPOINT,
                                 LUBM_ONTOLOGY_ENDPOINT_TYPE, properties_dict, QUERY_LIMIT)

In [43]:
lubm_graph_words_encoder = GraphWordsEncoder(properties_dict, properties_group_dict, classes_dict)

In [44]:
input_graph_encoding, resources_dictionary = lubm_graph_words_encoder.encode_graph(input_graph)

http://www.Department0.University0.edu http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://swat.cse.lehigh.edu/onto/univ-bench.owl#Department
-1 1 1
http://www.Department0.University0.edu/FullProfessor7 http://swat.cse.lehigh.edu/onto/univ-bench.owl#headOf http://www.Department0.University0.edu
-1 2 -2
http://www.Department0.University0.edu/GraduateStudent0 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-3 3 -2
http://www.Department0.University0.edu/GraduateStudent1 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-4 3 -2
http://www.Department0.University0.edu/GraduateStudent10 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-5 3 -2
http://www.Department0.University0.edu/GraduateStudent100 http://swat.cse.lehigh.edu/onto/univ-bench.owl#memberOf http://www.Department0.University0.edu
-6 3 -2
http://www.Department0.University0.edu/GraduateStudent

In [46]:
input_graph_encoding

{1: [(-1, 1)],
 2: [(-1, -2)],
 3: [(-3, -2),
  (-4, -2),
  (-5, -2),
  (-6, -2),
  (-7, -2),
  (-8, -2),
  (-9, -2),
  (-10, -2),
  (-11, -2),
  (-12, -2),
  (-13, -2),
  (-14, -2),
  (-15, -2),
  (-16, -2),
  (-17, -2),
  (-18, -2),
  (-19, -2),
  (-20, -2),
  (-21, -2),
  (-22, -2),
  (-23, -2),
  (-24, -2),
  (-25, -2),
  (-26, -2),
  (-27, -2),
  (-28, -2),
  (-29, -2),
  (-30, -2),
  (-31, -2),
  (-32, -2),
  (-33, -2),
  (-34, -2),
  (-35, -2),
  (-36, -2),
  (-37, -2),
  (-38, -2),
  (-39, -2),
  (-40, -2),
  (-41, -2),
  (-42, -2),
  (-43, -2),
  (-44, -2),
  (-45, -2),
  (-46, -2),
  (-47, -2),
  (-48, -2),
  (-49, -2),
  (-50, -2),
  (-51, -2),
  (-52, -2),
  (-53, -2),
  (-54, -2),
  (-55, -2),
  (-56, -2),
  (-57, -2),
  (-58, -2),
  (-59, -2),
  (-60, -2),
  (-61, -2),
  (-62, -2),
  (-63, -2),
  (-64, -2),
  (-65, -2),
  (-66, -2),
  (-67, -2),
  (-68, -2),
  (-69, -2),
  (-70, -2),
  (-71, -2),
  (-72, -2),
  (-73, -2),
  (-74, -2),
  (-75, -2),
  (-76, -2),
  (-77, -2)